In [ ]:
!pip install requests beautifulsoup4 pandas lxml

In [ ]:
# LG전자 지원 페이지 크롤링 코드
# PyCharm 주피터 노트북 환경

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from urllib.parse import urljoin, urlparse
import re

class LGSupportCrawler:
    def __init__(self):
        self.base_url = "https://www.lge.co.kr"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

    def get_page_content(self, url):
        """페이지 내용을 가져오는 함수 (디버깅 정보 포함)"""
        try:
            print(f"페이지 요청 중: {url}")
            response = self.session.get(url, timeout=15)

            print(f"응답 상태 코드: {response.status_code}")
            print(f"응답 헤더 Content-Type: {response.headers.get('Content-Type', 'Unknown')}")
            print(f"응답 내용 길이: {len(response.text)} 문자")

            response.raise_for_status()

            # 응답 내용의 일부를 확인
            if len(response.text) > 0:
                print("응답 내용 미리보기:")
                print(response.text[:500] + "..." if len(response.text) > 500 else response.text)
                print("=" * 50)

            return response.text

        except requests.RequestException as e:
            print(f"페이지 요청 실패: {e}")
            return None

    def parse_support_page(self, html_content):
        """지원 페이지를 파싱하는 함수 (디버깅 정보 포함)"""
        if not html_content:
            print("HTML 내용이 비어있습니다.")
            return []

        soup = BeautifulSoup(html_content, 'html.parser')
        support_items = []

        # 페이지 구조 분석을 위한 디버깅 정보
        print("=== 페이지 구조 분석 ===")
        print(f"페이지 제목: {soup.title.get_text() if soup.title else '제목 없음'}")

        # 더 광범위한 선택자로 시도
        selectors = [
            'div[class*="list"]',
            'ul[class*="list"]',
            'div[class*="item"]',
            'li[class*="item"]',
            'div[class*="solution"]',
            'div[class*="support"]',
            'div[class*="card"]',
            'article',
            '.list-item',
            '.solution-item',
            '.support-item'
        ]

        items_found = []
        for selector in selectors:
            elements = soup.select(selector)
            if elements:
                print(f"선택자 '{selector}'로 {len(elements)}개 요소 발견")
                items_found.extend(elements)

        # 중복 제거
        unique_items = list(set(items_found))
        print(f"중복 제거 후 총 {len(unique_items)}개 항목")

        # 항목이 없으면 전체 구조 출력
        if not unique_items:
            print("항목을 찾을 수 없습니다. 페이지 구조를 분석합니다...")
            # 주요 div 요소들 확인
            divs = soup.find_all('div', limit=20)
            for i, div in enumerate(divs):
                classes = div.get('class', [])
                if classes:
                    print(f"div {i+1}: class={classes}")

            # 리스트 요소들 확인
            lists = soup.find_all(['ul', 'ol'], limit=10)
            for i, lst in enumerate(lists):
                classes = lst.get('class', [])
                if classes:
                    print(f"list {i+1}: class={classes}")

        # 각 항목 파싱
        for item in unique_items:
            try:
                # 다양한 방법으로 제목 추출
                title = None
                title_selectors = [
                    'h1, h2, h3, h4, h5, h6',
                    'a[href*="solution"]',
                    'a[href*="support"]',
                    '.title',
                    '.subject',
                    '.name',
                    'strong',
                    'span[class*="title"]'
                ]

                for selector in title_selectors:
                    title_elem = item.select_one(selector)
                    if title_elem:
                        title = title_elem.get_text(strip=True)
                        if title:
                            break

                if not title:
                    # 텍스트 내용에서 제목 추출 시도
                    text = item.get_text(strip=True)
                    if text and len(text) > 10:
                        title = text[:100] + "..." if len(text) > 100 else text

                # 링크 추출
                link = ""
                link_elem = item.find('a', href=True)
                if link_elem:
                    href = link_elem['href']
                    if href.startswith('http'):
                        link = href
                    else:
                        link = urljoin(self.base_url, href)

                # 설명 추출
                description = ""
                desc_selectors = [
                    'p',
                    '.desc',
                    '.description',
                    '.content',
                    '.summary',
                    'span[class*="desc"]'
                ]

                for selector in desc_selectors:
                    desc_elem = item.select_one(selector)
                    if desc_elem:
                        description = desc_elem.get_text(strip=True)
                        if description:
                            break

                # 날짜 추출
                date = ""
                date_selectors = [
                    'time',
                    '.date',
                    '.time',
                    'span[class*="date"]',
                    'span[class*="time"]'
                ]

                for selector in date_selectors:
                    date_elem = item.select_one(selector)
                    if date_elem:
                        date = date_elem.get_text(strip=True)
                        if date:
                            break

                # 유효한 항목인지 확인
                if title and len(title.strip()) > 0:
                    support_items.append({
                        'title': title,
                        'description': description,
                        'link': link,
                        'date': date
                    })
                    print(f"항목 추가: {title[:50]}...")

            except Exception as e:
                print(f"항목 파싱 중 오류: {e}")
                continue

        print(f"총 {len(support_items)}개 항목 파싱 완료")
        return support_items

    def crawl_support_solutions(self, url):
        """지원 솔루션 페이지를 크롤링하는 메인 함수"""
        print(f"크롤링 시작: {url}")

        html_content = self.get_page_content(url)
        if not html_content:
            return []

        support_items = self.parse_support_page(html_content)

        # 페이지네이션 처리 (필요시)
        soup = BeautifulSoup(html_content, 'html.parser')
        pagination = soup.find(['div', 'nav'], class_=re.compile(r'pag|page'))

        if pagination:
            page_links = pagination.find_all('a', href=True)
            for page_link in page_links:
                if re.search(r'page=\d+', page_link['href']):
                    page_url = urljoin(self.base_url, page_link['href'])
                    print(f"다음 페이지 크롤링: {page_url}")

                    time.sleep(1)  # 서버 부하 방지
                    page_content = self.get_page_content(page_url)
                    if page_content:
                        page_items = self.parse_support_page(page_content)
                        support_items.extend(page_items)

        return support_items

    def save_to_csv(self, data, filename='lg_support_data.csv'):
        """데이터를 CSV 파일로 저장"""
        if data:
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"데이터가 {filename}에 저장되었습니다.")
        else:
            print("저장할 데이터가 없습니다.")

    def save_to_json(self, data, filename='lg_support_data.json'):
        """데이터를 JSON 파일로 저장"""
        if data:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"데이터가 {filename}에 저장되었습니다.")
        else:
            print("저장할 데이터가 없습니다.")

# 사용 예시 (디버깅 모드)
def main():
    # 크롤러 인스턴스 생성
    crawler = LGSupportCrawler()

    # 대상 URL (드럼세탁기 지원 페이지)
    target_url = "https://www.lge.co.kr/support/solutions?category=CT50019274&subCategory=CT50019309&categoryNm=%EC%84%B8%ED%83%81%EA%B8%B0/%EA%B1%B4%EC%A1%B0%EA%B8%B0/%EC%9D%98%EB%A5%98%EA%B4%80%EB%A6%AC%EA%B8%B0&subCategoryNm=%EB%93%9C%EB%9F%BC%EC%84%B8%ED%83%81%EA%B8%B0&mktModelCd=F0Q8CSVKW9.AKOR&sort=update&page=1&seq=0&sympCodeThree=1RRRTRT"

    # 단순한 URL로도 테스트
    simple_url = "https://www.lge.co.kr/support/solutions"

    print("=== 원본 URL 테스트 ===")
    support_data = crawler.crawl_support_solutions(target_url)

    # 원본 URL이 실패하면 단순 URL로 시도
    if not support_data:
        print("\n=== 단순 URL로 재시도 ===")
        support_data = crawler.crawl_support_solutions(simple_url)

    # 결과 출력
    print(f"\n총 {len(support_data)}개의 지원 항목을 수집했습니다.")

    # 처음 5개 항목 미리보기
    if support_data:
        print("\n=== 수집된 데이터 미리보기 ===")
        for i, item in enumerate(support_data[:5], 1):
            print(f"{i}. {item['title']}")
            if item['description']:
                print(f"   설명: {item['description'][:100]}...")
            print(f"   링크: {item['link']}")
            print(f"   날짜: {item['date']}")
            print("-" * 50)

    # 데이터 저장
    if support_data:
        crawler.save_to_csv(support_data)
        crawler.save_to_json(support_data)
    else:
        print("\n=== 문제 해결 방법 ===")
        print("1. 네트워크 연결 확인")
        print("2. URL이 올바른지 확인")
        print("3. 웹사이트가 JavaScript로 동적 로딩하는지 확인")
        print("4. 웹사이트의 robots.txt 확인")
        print("5. User-Agent 헤더 변경 시도")

    return support_data

# 추가 디버깅 함수
def debug_page_structure(url):
    """페이지 구조를 자세히 분석하는 디버깅 함수"""
    crawler = LGSupportCrawler()
    html_content = crawler.get_page_content(url)

    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')

        print("=== 페이지 구조 상세 분석 ===")
        print(f"전체 HTML 길이: {len(html_content)}")

        # 주요 태그들 개수 확인
        tags_to_check = ['div', 'ul', 'li', 'article', 'section', 'h1', 'h2', 'h3', 'a']
        for tag in tags_to_check:
            count = len(soup.find_all(tag))
            print(f"{tag} 태그 개수: {count}")

        # JavaScript 확인
        scripts = soup.find_all('script')
        print(f"JavaScript 태그 개수: {len(scripts)}")

        # 메타 태그 확인
        meta_tags = soup.find_all('meta')
        for meta in meta_tags:
            if meta.get('name') or meta.get('property'):
                print(f"Meta: {meta.get('name', meta.get('property', 'unknown'))}")

        # CSS 클래스 분석
        all_elements = soup.find_all(True)
        classes = set()
        for element in all_elements:
            if element.get('class'):
                classes.update(element.get('class'))

        print(f"사용된 CSS 클래스 개수: {len(classes)}")
        relevant_classes = [cls for cls in classes if any(keyword in cls.lower() for keyword in ['list', 'item', 'solution', 'support', 'card'])]
        print(f"관련 CSS 클래스: {relevant_classes[:10]}")

    return html_content

# 주피터 노트북에서 실행
if __name__ == "__main__":
    # 필요한 라이브러리 설치 (주피터 노트북에서 실행)
    # !pip install requests beautifulsoup4 pandas lxml

    # 크롤링 실행
    data = main()

    # 데이터 분석 예시
    if data:
        df = pd.DataFrame(data)
        print(f"\n=== 데이터 분석 결과 ===")
        print(f"총 항목 수: {len(df)}")
        print(f"링크가 있는 항목: {len(df[df['link'] != ''])}")
        print(f"설명이 있는 항목: {len(df[df['description'] != ''])}")

        # 제목 길이 분석
        if len(df) > 0:
            df['title_length'] = df['title'].str.len()
            print(f"평균 제목 길이: {df['title_length'].mean():.1f}자")
            print(f"가장 긴 제목: {df['title_length'].max()}자")
            print(f"가장 짧은 제목: {df['title_length'].min()}자")